# Curation of BiGG ids

Notebook for correcting misc. issues with BiGG dictionaries provided by @snmendoz in PR #188.

## 1. Splitting list into separate lists

We will start by splitting the original met/reaction files into 2 separates based on if they belong or not to BiGG:

In [1]:
# Function for splitting the file in two:
import csv
import os
def split_in_two(old_file_name, bigg_file_name):
    # Create list with BiGG ids:
    with open(bigg_file_name) as bigg_file:
        bigg_list = bigg_file.read().splitlines()
        
        
    # Split file in two:
    in_file_name = old_file_name.replace(".csv","_in.csv")
    out_file_name = old_file_name.replace(".csv","_newIDs.csv")
    with open(old_file_name) as old_file:
        with open(in_file_name, 'w', newline='') as in_file:
            with open(out_file_name, 'w', newline='') as out_file:
                old_reader = csv.reader(old_file, delimiter=',')
                in_writer = csv.writer(in_file, delimiter=',')
                out_writer = csv.writer(out_file, delimiter=',')
                for row in old_reader:
                    if row[1] in bigg_list:
                        in_writer.writerow([row[0], row[1]])
                    else:
                        out_writer.writerow([row[0], row[1]])

    # Replace file:
    os.remove(old_file_name)
    os.rename(in_file_name, old_file_name)

# Split both metabolite and reaction files:
split_in_two('../../ComplementaryData/databases/BiGGmetDictionary.csv', '../../ComplementaryData/databases/mets_already_in_bigg.txt')
split_in_two('../../ComplementaryData/databases/BiGGrxnDictionary.csv', '../../ComplementaryData/databases/rxns_already_in_bigg.txt')